<a href="https://colab.research.google.com/github/armarmarm927/dl_lecture_competition_pub/blob/main/sparkproto4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.init as init
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset,DataLoader
from torch.optim import lr_scheduler
import torchvision.datasets as dsets
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
batch_size = 100
train_data = dsets.CIFAR10(root='./tmp/cifar-10',train=True,download=True,transform=transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),transforms.RandomErasing(p=0.5,scale=(0.02,0.33),ratio=(0.3,3.3),value=0,inplace=False)]))
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_data = dsets.CIFAR10(root='./tmp/cifar-10',train=False,download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])]))
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=False)

100%|██████████| 170M/170M [00:19<00:00, 8.70MB/s]


Extracting ./tmp/cifar-10/cifar-10-python.tar.gz to ./tmp/cifar-10
Files already downloaded and verified


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model_ft = models.resnet50(pretrained=True)
model_ft.fc = nn.Linear(model_ft.fc.in_features,10)
net = model_ft.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 172MB/s]


In [5]:
criterion =nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.1,momentum=0.9,weight_decay=0.00005)
scheduler = lr_scheduler.LambdaLR(optimizer,lr_lambda=lambda epoch:0.95**epoch)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import numpy as np

loss, epoch_loss, count = 0, 0, 0
acc_list = []
loss_list = []
train_acc_list = []

for i in range(50):
    net.train()
    correct_train = 0
    total_train = 0

    for j, data in enumerate(train_loader, 0):
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        count += 1

        _, predicted = torch.max(outputs.data, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = 100. * correct_train / total_train
    train_acc_list.append(train_accuracy)
    print('%depoch: mean_loss: %.3f, Train Accuracy: %.2f%%' % (i+1, epoch_loss/count, train_accuracy))

    loss_list.append(epoch_loss/count)
    epoch_loss = 0
    count = 0
    correct = 0
    total = 0
    accuracy = 0.0

    # 混同行列用のリスト
    y_true = []
    y_pred = []

    net.eval()
    with torch.no_grad():
        for j, data in enumerate(test_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)

            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # 混同行列用データ保存
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = 100. * correct / total
    acc_list.append(accuracy)
    print('epoch:%d Test Accuracy(%d/%d): %.2f%%' % (i+1, correct, total, accuracy))

    # 混同行列の計算と表示
    cm = confusion_matrix(y_true, y_pred)
    print(f'Confusion Matrix (Epoch {i+1}):')
    print(cm)

    torch.save(net.state_dict(), 'Weight' + str(i+1))
    scheduler.step()


1epoch: mean_loss: 2.627, Train Accuracy: 20.30%
epoch:1 Test Accuracy(2694/10000): 26.94%
Confusion Matrix (Epoch 1):
[[366  54  12  28  26  60  26  27 361  40]
 [ 82 197   9  35  39 179  82  35 236 106]
 [103  28  73  50  94 189 290  25 117  31]
 [ 45  53  24  95  76 298 224  49  86  50]
 [ 51  34  63  34 150 125 398  25  84  36]
 [ 32  33  26  84  46 440 179  43  77  40]
 [ 14  32  62  56  64 134 529  19  39  51]
 [ 45  55  17  60  85 239 106  53 153 187]
 [217  66   5  13  24  67   5  14 530  59]
 [ 63  84  10  32  30 104  53  41 322 261]]
2epoch: mean_loss: 1.958, Train Accuracy: 27.92%
epoch:2 Test Accuracy(3251/10000): 32.51%
Confusion Matrix (Epoch 2):
[[288  41  63  15  15  30  25  36 384 103]
 [ 54 283  25  36  22  53  83  83 132 229]
 [ 73  26 266  75  42  99 237  63  86  33]
 [ 36  33 104 163  43 186 257  64  44  70]
 [ 18  15 234  48 115  61 335  80  59  35]
 [ 18  21  98 140  46 375 180  58  24  40]
 [ 11  13 179  75  38  66 528  38  12  40]
 [ 26  31  55  74  50 158 148 

KeyboardInterrupt: 

In [ ]:
print(acc_list)

In [ ]:
plt.plot(acc_list)
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,50,5))
plt.ylabel('Accuracy')
plt.show()